In [ ]:
import requests
from tabulate importpip tabulate

def test_investments_by_country(slug: str):
    """
    Test the investments by country API endpoint
    :param slug: The fund slug (e.g., 'coinbase-ventures')
    """
    base_url = "https://api.cryptorank.io/v0"
    endpoint = f"{base_url}/fund-chart/{slug}/investments-count-by-country"
    
    try:
        print(f"🔍 Testing investments by country for: {slug}")
        response = requests.get(endpoint, timeout=10)
        response.raise_for_status()
        
        json_data = response.json()
        print("✅ Raw API Response Received")
        
        if 'data' not in json_data:
            print("❌ No 'data' field in response")
            return None
        
        return process_country_data(json_data['data'])
        
    except requests.exceptions.RequestException as e:
        print(f"❌ API Error: {str(e)}")
        return None
    except Exception as e:
        print(f"❌ Processing Error: {str(e)}")
        return None

def process_country_data(country_data: list) -> dict:
    """Process and format the country investment data"""
    processed = {
        'countries': [],
        'total_normal': 0,
        'total_lead': 0
    }
    
    for country in country_data:
        # Skip if country structure is invalid
        if not isinstance(country, dict) or 'country' not in country:
            continue
            
        try:
            country_info = {
                'name': country['country'].get('name', 'Unknown'),
                'code': country['country'].get('code', 'XX'),
                'normal': country.get('normalCount', 0),
                'lead': country.get('leadCount', 0),
                'categories': []
            }
            
            processed['total_normal'] += country_info['normal']
            processed['total_lead'] += country_info['lead']
            
            # Ensure categories field is handled properly
            categories = country.get('categories', [])
            if isinstance(categories, list):  # Ensure categories is a list
                for category in categories:
                    if isinstance(category, dict):  # Ensure category is a dictionary
                        country_info['categories'].append({
                            'name': category.get('name', 'Unknown'),
                            'count': category.get('count', 0),
                            'percent': round(float(category.get('percent', 0)) * 100, 2)
                        })
            
            processed['countries'].append(country_info)
        except Exception as e:
            print(f"⚠️ Error processing country data: {str(e)}")
            continue
    
    return processed

def display_results(data):
    """Display the processed results"""
    if not data or not data.get('countries'):
        print("❌ No valid country data to display")
        return
    
    print(f"\n🌍 Total Investments: {data['total_normal']} (Normal) + {data['total_lead']} (Lead)")
    
    # Display top countries table
    headers = ["Rank", "Country", "Code", "Investments", "Lead", "% of Total"]
    table_data = []
    
    sorted_countries = sorted(data['countries'], key=lambda x: x['normal'], reverse=True)
    total_all = data['total_normal'] + data['total_lead']
    
    for idx, country in enumerate(sorted_countries[:15], 1):
        total_investments = country['normal'] + country['lead']
        percent_of_total = (total_investments / total_all) * 100 if total_all > 0 else 0
        
        table_data.append([idx, country['name'], country['code'], f"{country['normal']}", f"{country['lead']}", f"{percent_of_total:.1f}%"])
    
    print("\n🏆 Top Countries by Investment Count:")
    print(tabulate(table_data, headers=headers, tablefmt="pretty"))
    
    # Display category breakdown for top countries
    print("\n📊 Category Breakdown for Top Countries:")
    for country in sorted_countries[:5]:
        if country['categories']:
            print(f"\n{country['name']} ({country['code']}):")
            cat_headers = ["Category", "Count", "% of Country"]
            cat_table = []
            
            # Sort categories by percent and prepare them for display
            for cat in sorted(country['categories'], key=lambda x: x['percent'], reverse=True):
                cat_table.append([cat['name'], cat['count'], f"{cat['percent']}%"])
            
            print(tabulate(cat_table, headers=cat_headers, tablefmt="pretty"))

if __name__ == "__main__":
    # Test with Coinbase Ventures
    print("=== Testing Investments by Country API ===")
    test_slugs = [
        "coinbase-ventures",
        # Add more test slugs here if needed
    ]
    
    for slug in test_slugs:
        country_data = test_investments_by_country(slug)
        if country_data:
            display_results(country_data)
            print("\n" + "="*50 + "\n")
    
    print("=== Test Complete ===")


ModuleNotFoundError: No module named 'tabulate'